In [4]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt
import seaborn as sns ; sns.set()


load_dotenv('BDD_URL.env')
BDD_URL = os.environ['BDD_URL']
engine = create_engine(BDD_URL)



In [5]:
SQL = "SELECT * from principal.title_ratings "
basics_df = pd.read_sql(SQL, engine)

engine.dispose()



In [6]:
basics_df[basics_df["averageRating"]>9]

,tconst,averageRating,numVotes
1147,tt0002582,9.1,18
2375,tt0008528,9.3,15
12047,tt0027944,9.4,7
13293,tt0029523,9.5,10
18577,tt0035924,9.1,9
...,...,...,...
1398111,tt9915084,9.2,27
1398133,tt9915898,9.4,3248
1398159,tt9916400,9.4,6
1398164,tt9916460,9.4,18


In [7]:
SQL = "SELECT * from principal.title_basics limit 10000"
df = pd.read_sql(SQL, engine)

In [8]:
df.titleType.value_counts()

titleType
tvEpisode      4480
short          3196
movie          1204
video           588
tvMovie         231
tvSeries        199
tvShort          38
tvMiniSerie      33
tvSpecial        31
Name: count, dtype: int64

In [9]:
listclass=['decades1','decades2','decades3','decades4','decades5', 'decades6','decades7','decades8','decades9','decades10','decades11','decades12','decades13','decades14']
listvalues=[1900,1910,1920,1930,1940,1950,1960,1970,1980,1990,2000,2010,2020,2030,2040]
dff=pd.cut(df['startYear'], bins=listvalues, labels=listclass)
dff.value_counts()

startYear
decades12    7359
decades11    1100
decades9      735
decades10     294
decades7      211
decades8      166
decades6       44
decades4       19
decades5       10
decades1        9
decades13       8
decades2        6
decades3        5
decades14       0
Name: count, dtype: int64

In [10]:
SQL = '''SELECT "averageRating", "startYear", "runtimeMinutes" from principal.title_basics join principal.title_ratings on principal.title_basics."tconst" = principal.title_ratings."tconst"  limit 50000;'''
df = pd.read_sql(SQL, engine)
df



,averageRating,startYear,runtimeMinutes
0,8.1,2012,60.0
1,7.8,2012,63.0
2,8.1,2012,64.0
3,8.1,2012,60.0
4,7.6,2012,64.0
...,...,...,...
49995,7.6,2014,104.0
49996,6.0,2015,120.0
49997,6.6,2015,97.0
49998,6.3,2006,100.0


In [11]:
df["averageRating"].fillna(df["averageRating"].mean(), inplace=True)
df["startYear"].fillna(df["startYear"].mean(), inplace=True)
df["runtimeMinutes"].fillna(df["runtimeMinutes"].mean(), inplace=True)
df.describe()

,averageRating,startYear,runtimeMinutes
count,50000.000000,50000.000000,50000.000000
mean,6.995480,2011.358520,49.757503
std,1.356395,9.987417,196.914113
min,1.000000,1888.000000,1.000000
25%,6.300000,2013.000000,22.000000
50%,7.200000,2014.000000,43.000000
75%,7.900000,2014.000000,75.000000
max,10.000000,2024.000000,43200.000000


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   averageRating   50000 non-null  float64
 1   startYear       50000 non-null  int64  
 2   runtimeMinutes  50000 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 1.1 MB


In [13]:


y=df["averageRating"]
X=df[["startYear","runtimeMinutes"]]


In [14]:
# Import des librairies
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error
import numpy as np

# Création de la partition train / test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=42)


scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Déclaration du modèle
model = LinearRegression()
# Entrainement du modèle
model.fit(X_train, y_train)

# Prédiction sur nouvelles valeurs
y_pred = model.predict(X_test)

print('score =',model.score(X_train, y_train))

# Accès au score 
print('R2 =',r2_score(y_test, y_pred))

mse = mean_squared_error(y_test, y_pred)
print("MSE = ", mse)

rmse = np.sqrt(mse)
print("RMSE =", rmse)

# Calculer l'erreur absolue moyenne (MAE)
mae = mean_absolute_error(y_test, y_pred)
print("MAE =", mae)



score = 0.06912259183952407
R2 = -5.053080713663773
MSE =  11.286042202492062
RMSE = 3.359470524129072
MAE = 1.0320260300675197
